In [20]:
from ortools.sat.python import cp_model

In [10]:
import os
os.listdir()

from google.colab import files
uploaded = files.upload()

import pandas as pd
fname = next(iter(uploaded))
df = pd.read_csv(fname, encoding='utf-8-sig')
df.head()

Saving 학급반편성CSP 문제 입력파일.csv to 학급반편성CSP 문제 입력파일 (1).csv


,id,name,sex,score,24년 학급,클럽,좋은관계,나쁜관계,Leadership,Piano,비등교,운동선호
0,202501,Andy,boy,90,a,노래,202502.0,202503.0,NaN,NaN,NaN,NaN
1,202502,Briada,girl,51,b,댄스,NaN,NaN,NaN,yes,NaN,NaN
2,202503,Charlie,boy,91,c,야구,NaN,NaN,yes,NaN,NaN,yes
3,202504,Kevin W,boy,60,f,미술,NaN,202677.0,NaN,NaN,NaN,NaN
4,202505,Matthew G,boy,96,f,댄스,NaN,NaN,NaN,NaN,yes,NaN


In [11]:
import pandas as pd
from math import floor, ceil

df = pd.read_csv('학급반편성CSP 문제 입력파일.csv')
df.head()

,id,name,sex,score,24년 학급,클럽,좋은관계,나쁜관계,Leadership,Piano,비등교,운동선호
0,202501,Andy,boy,90,a,노래,202502.0,202503.0,NaN,NaN,NaN,NaN
1,202502,Briada,girl,51,b,댄스,NaN,NaN,NaN,yes,NaN,NaN
2,202503,Charlie,boy,91,c,야구,NaN,NaN,yes,NaN,NaN,yes
3,202504,Kevin W,boy,60,f,미술,NaN,202677.0,NaN,NaN,NaN,NaN
4,202505,Matthew G,boy,96,f,댄스,NaN,NaN,NaN,NaN,yes,NaN


In [12]:
df.info()
df.describe(include="all")

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 200 entries, 0 to 199
Data columns (total 12 columns):
 #   Column      Non-Null Count  Dtype  
---  ------      --------------  -----  
 0   id          200 non-null    int64  
 1   name        200 non-null    object 
 2   sex         200 non-null    object 
 3   score       200 non-null    int64  
 4   24년 학급      200 non-null    object 
 5   클럽          200 non-null    object 
 6   좋은관계        20 non-null     float64
 7   나쁜관계        20 non-null     float64
 8   Leadership  20 non-null     object 
 9   Piano       20 non-null     object 
 10  비등교         20 non-null     object 
 11  운동선호        20 non-null     object 
dtypes: float64(2), int64(2), object(8)
memory usage: 18.9+ KB


,id,name,sex,score,24년 학급,클럽,좋은관계,나쁜관계,Leadership,Piano,비등교,운동선호
count,200.000000,200,200,200.000000,200,200,20.000000,20.000000,20,20,20,20
unique,NaN,135,2,NaN,6,10,NaN,NaN,1,1,1,1
top,NaN,Robert M,boy,NaN,a,연극,NaN,NaN,yes,yes,yes,yes
freq,NaN,5,141,NaN,36,26,NaN,NaN,20,20,20,20
mean,202600.500000,NaN,NaN,78.930000,NaN,NaN,202630.150000,202604.700000,NaN,NaN,NaN,NaN
std,57.879185,NaN,NaN,11.745419,NaN,NaN,57.595938,68.645619,NaN,NaN,NaN,NaN
min,202501.000000,NaN,NaN,51.000000,NaN,NaN,202502.000000,202502.000000,NaN,NaN,NaN,NaN
25%,202550.750000,NaN,NaN,69.750000,NaN,NaN,202607.250000,202532.250000,NaN,NaN,NaN,NaN
50%,202600.500000,NaN,NaN,78.000000,NaN,NaN,202640.500000,202620.000000,NaN,NaN,NaN,NaN
75%,202650.250000,NaN,NaN,90.000000,NaN,NaN,202675.000000,202655.250000,NaN,NaN,NaN,NaN


In [13]:
# Boolean 컬럼 처리
def to_bool(s):
    return str(s).strip().lower() == 'yes'

for col in ['Leadership', 'Piano', '비등교', '운동선호']:
    df[col] = df[col].apply(to_bool)

# 관계 id 처리
def id_or_none(v):
    try:
        if pd.isna(v): return None
        return int(v)
    except:
        return None

df['좋은관계'] = df['좋은관계'].apply(id_or_none)
df['나쁜관계'] = df['나쁜관계'].apply(id_or_none)

# 전년도/클럽 결측값 채우기
df['24년 학급'] = df['24년 학급'].fillna('미지정').astype(str)
df['클럽'] = df['클럽'].fillna('미지정').astype(str)

df.head()

,id,name,sex,score,24년 학급,클럽,좋은관계,나쁜관계,Leadership,Piano,비등교,운동선호
0,202501,Andy,boy,90,a,노래,202502.0,202503.0,False,False,False,False
1,202502,Briada,girl,51,b,댄스,NaN,NaN,False,True,False,False
2,202503,Charlie,boy,91,c,야구,NaN,NaN,True,False,False,True
3,202504,Kevin W,boy,60,f,미술,NaN,202677.0,False,False,False,False
4,202505,Matthew G,boy,96,f,댄스,NaN,NaN,False,False,True,False


In [14]:
n = len(df)                       # 학생 수 = 200
num_classes = 6
class_size = [33,33,33,33,34,34]

id_to_idx = {int(r.id): i for i, r in df.iterrows()}

# 균등 분배 범위 함수
def bounds(total, k=num_classes):
    return floor(total/k), ceil(total/k)

In [15]:
model = cp_model.CpModel()

# x[i][c] : 학생 i가 반 c에 배정되면 1
x = [[model.NewBoolVar(f'x_{i}_{c}') for c in range(num_classes)] for i in range(n)]

# (기본) 각 학생은 정확히 한 반
for i in range(n):
    model.Add(sum(x[i][c] for c in range(num_classes)) == 1)

# 각 반 정원 (33,33,33,33,34,34)
for c in range(num_classes):
    model.Add(sum(x[i][c] for i in range(n)) == class_size[c])

In [16]:
# (1) 문제 아동 배치
for i, row in df.iterrows():
    bad_id = row['나쁜관계']
    if bad_id is not None and bad_id in id_to_idx:
        j = id_to_idx[bad_id]
        for c in range(num_classes):
            model.Add(x[i][c] + x[j][c] <= 1)

    if row['비등교']:
        good_id = row['좋은관계']
        if good_id is not None and good_id in id_to_idx:
            j = id_to_idx[good_id]
            for c in range(num_classes):
                model.Add(x[i][c] == x[j][c])

# (2) 리더십 최소 1명
leaders = df.index[df['Leadership']].tolist()
for c in range(num_classes):
    model.Add(sum(x[i][c] for i in leaders) >= 1)

# (3) 피아노 균등
pianos = df.index[df['Piano']].tolist()
lb, ub = bounds(len(pianos))
for c in range(num_classes):
    model.Add(sum(x[i][c] for i in pianos) >= lb)
    model.Add(sum(x[i][c] for i in pianos) <= ub)

# (4) 성적 균형
scores = df['score'].astype(int).tolist()
target = sum(scores) // num_classes
class_score = [model.NewIntVar(0, 100*40, f'score_{c}') for c in range(num_classes)]
for c in range(num_classes):
    model.Add(class_score[c] == sum(scores[i]*x[i][c] for i in range(n)))

dev = [model.NewIntVar(0, 100*40, f'dev_{c}') for c in range(num_classes)]
for c in range(num_classes):
    tmp = model.NewIntVar(-100*40, 100*40, f'diff_{c}')
    model.Add(tmp == class_score[c] - target)
    model.AddAbsEquality(dev[c], tmp)
dev_max = model.NewIntVar(0, 100*40, 'dev_max')
model.AddMaxEquality(dev_max, dev)

# (5) 비등교 균등
at_risk = df.index[df['비등교']].tolist()
lb, ub = bounds(len(at_risk))
for c in range(num_classes):
    model.Add(sum(x[i][c] for i in at_risk) >= lb)
    model.Add(sum(x[i][c] for i in at_risk) <= ub)

# (6) 남녀 균등
boys = df.index[df['sex']=='boy'].tolist()
girls= df.index[df['sex']=='girl'].tolist()
lb_b, ub_b = bounds(len(boys))
lb_g, ub_g = bounds(len(girls))
for c in range(num_classes):
    model.Add(sum(x[i][c] for i in boys) >= lb_b)
    model.Add(sum(x[i][c] for i in boys) <= ub_b)
    model.Add(sum(x[i][c] for i in girls) >= lb_g)
    model.Add(sum(x[i][c] for i in girls) <= ub_g)

# (7) 운동 균등
sports = df.index[df['운동선호']].tolist()
lb, ub = bounds(len(sports))
for c in range(num_classes):
    model.Add(sum(x[i][c] for i in sports) >= lb)
    model.Add(sum(x[i][c] for i in sports) <= ub)

# (8) 전년도 학급 분산
for g, group_df in df.groupby('24년 학급'):
    idxs = group_df.index.tolist()
    lb, ub = bounds(len(idxs))
    for c in range(num_classes):
        model.Add(sum(x[i][c] for i in idxs) >= lb)
        model.Add(sum(x[i][c] for i in idxs) <= ub)

# (9) 클럽 분산
for g, group_df in df.groupby('클럽'):
    idxs = group_df.index.tolist()
    lb, ub = bounds(len(idxs))
    for c in range(num_classes):
        model.Add(sum(x[i][c] for i in idxs) >= lb)
        model.Add(sum(x[i][c] for i in idxs) <= ub)

# 목적함수: 성적 편차 최소화
model.Minimize(dev_max)

In [17]:
solver = cp_model.CpSolver()
solver.parameters.max_time_in_seconds = 60
solver.parameters.num_search_workers = 8

res = solver.Solve(model)
print("Status:", solver.StatusName(res))

Status: FEASIBLE


In [18]:
assign = []
for i in range(n):
    c = max(range(num_classes), key=lambda k: solver.Value(x[i][k]))
    assign.append(c)

df_out = df.copy()
df_out['new_class'] = [a+1 for a in assign]

# 반별 요약
def cnt(mask, cls):
    idxs = df.index[mask].tolist()
    return sum(solver.Value(x[i][cls]) for i in idxs)

print("\n=== 반별 요약 ===")
for c in range(num_classes):
    male = cnt(df['sex']=='boy', c)
    female = cnt(df['sex']=='girl', c)
    lead = cnt(df['Leadership'], c)
    piano= cnt(df['Piano'], c)
    risk = cnt(df['비등교'], c)
    sport= cnt(df['운동선호'], c)
    sc = solver.Value(class_score[c])
    print(f'반 {c+1}: 인원 {class_size[c]}, 남 {male}, 여 {female}, 리더 {lead}, 피아노 {piano}, 위험 {risk}, 운동 {sport}, 총점 {sc}')

# CSV 저장
df_out.to_csv('class_assignment.csv', index=False)
print("\n저장 완료: class_assignment.csv")


=== 반별 요약 ===
반 1: 인원 33, 남 23, 여 10, 리더 1, 피아노 4, 위험 3, 운동 3, 총점 2632
반 2: 인원 33, 남 23, 여 10, 리더 5, 피아노 3, 위험 3, 운동 4, 총점 2630
반 3: 인원 33, 남 23, 여 10, 리더 5, 피아노 3, 위험 3, 운동 3, 총점 2631
반 4: 인원 33, 남 24, 여 9, 리더 3, 피아노 4, 위험 4, 운동 3, 총점 2631
반 5: 인원 34, 남 24, 여 10, 리더 3, 피아노 3, 위험 4, 운동 4, 총점 2632
반 6: 인원 34, 남 24, 여 10, 리더 3, 피아노 3, 위험 3, 운동 3, 총점 2630

저장 완료: class_assignment.csv


In [19]:
# === 자동 컬럼탐지 + 검증 셀 (KeyError: 'class' 수정) ===
import pandas as pd
import numpy as np
from math import ceil

def pick_col(cols, candidates):
    for c in candidates:
        if c in cols:
            return c
    return None

# 1) 데이터 로드
df = pd.read_csv("학급반편성CSP 문제 입력파일.csv")
assign = pd.read_csv("class_assignment.csv")

if "new_class" in assign.columns:
    assign = assign.rename(columns={"new_class": "class"})

# 2) id / class 컬럼 자동 탐지
id_in_df     = pick_col(df.columns,     ["id","ID","Id","학생ID","student_id"])
id_in_assign = pick_col(assign.columns, ["id","ID","Id","학생ID","student_id"])
class_in_assign = pick_col(assign.columns, ["class","Class","반","반번호","반_번호","class_id","classId"])

# id가 인덱스에 있을 수 있으니 보정
if id_in_assign is None and assign.index.name in ["id","ID","Id"]:
    assign = assign.reset_index()
    id_in_assign = pick_col(assign.columns, ["id","ID","Id"])

if id_in_df is None or id_in_assign is None or class_in_assign is None:
    raise ValueError(
        f"[컬럼 확인 필요]\n"
        f"- 원본 df.columns: {df.columns.tolist()}\n"
        f"- 배정 assign.columns: {assign.columns.tolist()}\n"
        f"위 목록에서 id 또는 class(반) 컬럼명을 하나로 정해주세요."
    )

# 3) 컬럼 표준화 후 머지
df = df.rename(columns={id_in_df: "id"})
assign = assign.rename(columns={id_in_assign: "id", class_in_assign: "class"})
df = df.merge(assign[["id","class"]], on="id", how="inner")

num_classes = df["class"].nunique()
sizes = df["class"].value_counts().sort_index()
print("반별 인원:", dict(sizes))

def ok(msg, cond, detail=None):
    print(("✅ " if cond else "❌ ") + msg)
    if (not cond) and detail is not None:
        print(detail)

all_ok = True

# (0) 반 인원: 33×4, 34×2
size_ok = sorted(sizes.tolist()) == sorted([33,33,33,33,34,34])
ok("반 인원: 33×4, 34×2", size_ok, sizes)
all_ok &= size_ok

# (1-A) 문제아동(나쁜관계) 페어 같은 반 금지
bad_pairs = []
if "나쁜관계" in df.columns:
    for _, r in df.dropna(subset=["나쁜관계"]).iterrows():
        try:
            other = int(r["나쁜관계"])
            same = int(df.loc[df["id"]==other, "class"].iloc[0]) == int(r["class"])
            if same: bad_pairs.append((int(r["id"]), other, int(r["class"])))
        except Exception:
            pass
ok("문제아동-상대(나쁜관계) 같은 반 금지", len(bad_pairs)==0, bad_pairs if bad_pairs else None)
all_ok &= (len(bad_pairs)==0)

# (1-B) 비등교 위험 아동은 '좋은관계' 친구와 같은 반
need_pairs, miss_pairs = [], []
if {"비등교","좋은관계"}.issubset(df.columns):
    needy = df[df["비등교"].astype(str).str.lower().eq("yes")]
    for _, r in needy.iterrows():
        if pd.isna(r["좋은관계"]):
            miss_pairs.append(int(r["id"]))
        else:
            try:
                g = int(r["좋은관계"])
                same = int(df.loc[df["id"]==g, "class"].iloc[0]) == int(r["class"])
                if not same: need_pairs.append((int(r["id"]), g, int(r["class"])))
            except Exception:
                miss_pairs.append(int(r["id"]))
cond_1B = (len(need_pairs)==0)
ok("비등교 위험 아동이 보호 친구와 같은 반", cond_1B, need_pairs if need_pairs else None)
if miss_pairs:
    print("ℹ️ 보호 친구 미지정(공란):", miss_pairs)
all_ok &= cond_1B

# (2) 각 반 리더 ≥ 1
missing_leader = []
if "Leadership" in df.columns:
    leaders = df[df["Leadership"].astype(str).str.lower().eq("yes")].groupby("class")["id"].count()
    missing_leader = [c for c in sorted(df["class"].unique()) if leaders.get(c,0)<1]
ok("각 반 리더 최소 1명", len(missing_leader)==0, missing_leader if missing_leader else None)
all_ok &= (len(missing_leader)==0)

# (3) 피아노 균등(차이 ≤ 1)
if "Piano" in df.columns:
    piano = df[df["Piano"].astype(str).str.lower().eq("yes")].groupby("class")["id"].count()
    piano = piano.reindex(sorted(df["class"].unique()), fill_value=0)
    ok("피아노 인원 균등(차이 ≤ 1)", piano.max()-piano.min()<=1, dict(piano))
    all_ok &= (piano.max()-piano.min()<=1)

# (4) 성적 평균 범위 ≤ 5 (필요시 기준 조정)
if "score" in df.columns:
    avg = df.groupby("class")["score"].mean()
    spread = avg.max()-avg.min()
    ok(f"성적 균형(반 평균 범위 ≤ 5) | 실제={spread:.2f}", spread<=5, avg)
    all_ok &= (spread<=5)

# (5) 위험(문제) 아동 수 균등(차이 ≤ 1) – 나쁜관계 notna 카운트
if "나쁜관계" in df.columns:
    risk = df.index[df['나쁜관계'].notna()].tolist()   # 위험 아동 id
    lb, ub = bounds(len(risk))  # floor/ceil 분배
    for c in range(num_classes):
        model.Add(sum(x[i][c] for i in risk) >= lb)
        model.Add(sum(x[i][c] for i in risk) <= ub)

# (6) 남자 수 균등(차이 ≤ 1)
if "sex" in df.columns:
    boys = df[df["sex"].astype(str).str.lower().eq("boy")].groupby("class")["id"].count()
    boys = boys.reindex(sorted(df["class"].unique()), fill_value=0)
    ok("남자 수 균등(차이 ≤ 1)", boys.max()-boys.min()<=1, dict(boys))
    all_ok &= (boys.max()-boys.min()<=1)

# (7) 운동 선호 균등(차이 ≤ 1)
if "운동선호" in df.columns:
    sport = df[df["운동선호"].astype(str).str.lower().eq("yes")].groupby("class")["id"].count()
    sport = sport.reindex(sorted(df["class"].unique()), fill_value=0)
    ok("운동 선호 인원 균등(차이 ≤ 1)", sport.max()-sport.min()<=1, dict(sport))
    all_ok &= (sport.max()-sport.min()<=1)

# (8) 전년도 학급 동반 과밀 방지
if "24년 학급" in df.columns:
    ct = pd.crosstab(df["24년 학급"], df["class"])
    bad_cells = []
    for r in ct.index:
        n_r = int(ct.loc[r].sum())
        cap = ceil(n_r/num_classes)
        for c in ct.columns:
            v = int(ct.loc[r, c])
            if v>cap: bad_cells.append((r,c,v,cap))
    ok("전년도 동반 과밀 방지(각 셀 ≤ ceil(n_r/6))", len(bad_cells)==0, bad_cells if bad_cells else None)
    all_ok &= (len(bad_cells)==0)

# (9) 클럽 편향 방지(클럽별 분포 차이 ≤ 1)
club_ok = True
if "클럽" in df.columns:
    for club, sub in df.groupby("클럽"):
        cnt = sub.groupby("class")["id"].count().reindex(sorted(df["class"].unique()), fill_value=0)
        if cnt.max()-cnt.min()>1:
            club_ok = False
            print("❌ 클럽 편향:", club, dict(cnt))
ok("클럽 편향 방지(각 클럽 분포 차이 ≤ 1)", club_ok)
all_ok &= club_ok

print("\n=== 최종 판정 ===")
print("🎉 모든 제약 충족!" if all_ok else "⚠️ 일부 제약 위반이 있습니다. 위의 ❌ 항목을 확인하세요.")

반별 인원: {1: np.int64(33), 2: np.int64(33), 3: np.int64(33), 4: np.int64(33), 5: np.int64(34), 6: np.int64(34)}
✅ 반 인원: 33×4, 34×2
✅ 문제아동-상대(나쁜관계) 같은 반 금지
✅ 비등교 위험 아동이 보호 친구와 같은 반
ℹ️ 보호 친구 미지정(공란): [202505, 202515, 202523, 202537, 202545, 202555, 202559, 202563, 202637, 202641, 202654, 202673, 202676, 202678, 202682, 202684, 202689]
✅ 각 반 리더 최소 1명
✅ 피아노 인원 균등(차이 ≤ 1)
✅ 성적 균형(반 평균 범위 ≤ 5) | 실제=2.40
✅ 남자 수 균등(차이 ≤ 1)
✅ 운동 선호 인원 균등(차이 ≤ 1)
✅ 전년도 동반 과밀 방지(각 셀 ≤ ceil(n_r/6))
✅ 클럽 편향 방지(각 클럽 분포 차이 ≤ 1)

=== 최종 판정 ===
🎉 모든 제약 충족!
